In [95]:
import numpy as np
import pulp
from collections import Counter

def transform_Counter(C):
    C = Counter(C)
    for i in range(max(C)):
        if i not in C:
            C[i] = 0
    return C
    

def earth_movers_distance(P, Q, mass_penalty=1):
    P = transform_Counter(P)
    Q = transform_Counter(Q)
    m, n = len(P), len(Q)
    c = np.zeros((m, n))

    for i in range(m):
        for j in range(n):
            c[i][j] = abs(i-j)
           
    # Calculate total mass of P and Q
    total_P = sum(P.values())
    total_Q = sum(Q.values())
    
    model = pulp.LpProblem("EMD", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", [(i, j) for i in range(m) for j in range(n)], lowBound=0, cat="Continuous")
    for i in range(m):
        model += pulp.lpSum([x[(i, j)] for j in range(n)]) <= P[i]
    for j in range(n):
        model += pulp.lpSum([x[(i, j)] for i in range(m)]) <= Q[j]
        
    model+=pulp.lpSum([x[(i, j)] for j in range(n) for i in range(m) ]) == min(total_P, total_Q)
    
    model += pulp.lpSum([c[i][j] * x[(i, j)] for i in range(m) for j in range(n)])
    model.solve()
    print("\ndistance: {}".format(pulp.value(model.objective)))
    #for v in model.variables():
    #    print(v.name, "=", v.value())
    print("total_P: {}".format(total_P))
    print("total_Q: {}".format(total_Q))
    return pulp.value(model.objective) + abs(total_P - total_Q) * mass_penalty


In [96]:
def test_earth_movers_distance():
    # Similar histograms
    assert earth_movers_distance([1, 1, 2, 2, 3, 4, 5, 5, 6, 7, 8], [2, 4, 5, 3, 5, 1, 2, 6, 1, 7, 8]) == 0
    # Histograms with only forward movements
    assert earth_movers_distance([1, 1, 1, 3, 4, 4, 6], [1, 1, 1, 1, 3, 3, 4]) == 6
    assert earth_movers_distance([1, 1, 3, 3, 6, 6, 7], [1, 1, 2, 3, 4, 4, 5]) == 7
    # Histograms with forward/backward movements
    assert earth_movers_distance([1, 1, 1, 3, 4, 4, 6], [1, 1, 1, 1, 3, 8, 8]) == 9
    assert earth_movers_distance([1, 1, 3, 3, 3, 4, 7, 9], [1, 2, 2, 3, 3, 4, 8, 12]) == 6
    # Histograms with extra mass
    assert earth_movers_distance([1, 1, 1, 3, 3, 3, 5, 5, 5], [1, 1, 1, 3, 3, 3, 5, 5, 5, 6]) == 1
    assert earth_movers_distance([1, 1, 1, 3, 3, 3, 5, 5, 5], [1, 1, 1, 3, 3, 3, 4, 5, 5, 5]) == 1
    assert earth_movers_distance([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4], [1, 2, 3, 4]) == 12
    assert earth_movers_distance([1, 1, 1, 2, 4, 4, 5, 5, 5, 6, 6], [12, 12, 13, 14, 14, 14, 15, 16, 16, 16, 17]) == 119
    assert earth_movers_distance([1, 1, 1, 2, 4, 4, 5, 5, 5, 6, 6], [12, 12, 13, 14, 14, 14, 15, 16, 16, 16]) == 104


In [97]:
test_earth_movers_distance()


distance: 0.0
total_P: 11
total_Q: 11

distance: 6.0
total_P: 7
total_Q: 7

distance: 7.0
total_P: 7
total_Q: 7

distance: 9.0
total_P: 7
total_Q: 7

distance: 6.0
total_P: 8
total_Q: 8

distance: 0.0
total_P: 9
total_Q: 10

distance: 0.0
total_P: 9
total_Q: 10

distance: 0.0
total_P: 16
total_Q: 4

distance: 119.0
total_P: 11
total_Q: 11

distance: 103.0
total_P: 11
total_Q: 10


In [94]:
earth_movers_distance([1, 1, 1, 3, 4, 4, 6], [1, 1, 1, 1, 3, 3, 4])


distance: 6.0
x_(0,_0) = 0.0
x_(0,_1) = 0.0
x_(0,_2) = 0.0
x_(0,_3) = 0.0
x_(0,_4) = 0.0
x_(1,_0) = 0.0
x_(1,_1) = 3.0
x_(1,_2) = 0.0
x_(1,_3) = 0.0
x_(1,_4) = 0.0
x_(2,_0) = 0.0
x_(2,_1) = 0.0
x_(2,_2) = 0.0
x_(2,_3) = 0.0
x_(2,_4) = 0.0
x_(3,_0) = 0.0
x_(3,_1) = 1.0
x_(3,_2) = 0.0
x_(3,_3) = 0.0
x_(3,_4) = 0.0
x_(4,_0) = 0.0
x_(4,_1) = 0.0
x_(4,_2) = 0.0
x_(4,_3) = 1.0
x_(4,_4) = 1.0
x_(5,_0) = 0.0
x_(5,_1) = 0.0
x_(5,_2) = 0.0
x_(5,_3) = 0.0
x_(5,_4) = 0.0
x_(6,_0) = 0.0
x_(6,_1) = 0.0
x_(6,_2) = 0.0
x_(6,_3) = 1.0
x_(6,_4) = 0.0
total_P: 7
total_Q: 7


6.0

In [ ]:
[1, 2, 2, 1] = [1, 2, 3, 1]